In [1]:
from triplet import *
from data import *
from preprocess import *
from model import *

from keras.models import Model
from keras.layers import Lambda, Activation, Input, Conv2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np
from keras import backend as K
from keras.optimizers import Adam
from keras.layers import Input
from sklearn.preprocessing import normalize

Using TensorFlow backend.


In [2]:
root = './face'
name = 'Tran Thanh'
reader = LFWReader(dir_images=root,data_name = name)

In [3]:
def preprocess_input(batch_image):
    if not isinstance(batch_image, (np.ndarray, np.generic)):
        error_msg = "data must be 4d numpy array, but found {}"
        raise TypeError(error_msg.format(type(batch_image)))
    shape = batch_image.shape
    if len(shape) != 4:
        error_msg = "data must be shape of (batch, 224, 224, 3), but found {}"
        raise ValueError(error_msg.format(shape))
    (batch, size0, size1, channel) = shape
    if size0 != 224 or size1 != 224 or channel != 3:
        error_msg = "data must be shape of (batch, 224, 224, 3), but found {}"
        raise ValueError(error_msg.format(shape))
        
    batch_image = batch_image.astype(np.float32)
    batch_image = batch_image.transpose([0,3,1,2])
    batch_image = batch_image.transpose([0,2,3,1])
    return batch_image

In [4]:
def _ShowImg(img):
    plt.figure()
    plt.imshow(img.astype('uint8'))
    plt.show()
    plt.close()

def _ReadAndResize(filepath):
    im = Image.open((filepath)).convert('RGB')
    im = im.resize((224, 224))
    return np.array(im, dtype="float32")

def _Flip(im_array):
    if np.random.uniform(0, 1) > 0.7:
        im_array = np.fliplr(im_array)
    return im_array

In [6]:
list_pos = []
list_anchor = []
list_neg = []
for _ in range(batch_size):
    path_anchor, path_pos, path_neg = reader.GetTripletSingleID()
    img_anchor = _Flip(_ReadAndResize(path_anchor))
    img_pos = _Flip(_ReadAndResize(path_pos))
    img_neg = _Flip(_ReadAndResize(path_neg))
    list_pos.append(img_pos)
    list_anchor.append(img_anchor)
    list_neg.append(img_neg)
    print((np.array(img_pos)).shape)

(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)
(224, 224, 3)


In [ ]:
reader.GetTripletSingleID()

In [ ]:
list_anchor

In [ ]:
a.shape

In [ ]:
gen = TripletGeneratorSingleID(reader)
data=next(gen)

In [ ]:
def _TestTripletGenerator(reader):  
    gen = TripletGeneratorSingleID(reader)
    data = next(gen)
    imgs_anchor = data[0]['anchor_input']
    imgs_pos = data[0]['positive_input']
    imgs_neg = data[0]['negative_input']
    print(imgs_anchor.shape)
    print(imgs_pos.shape)
    print(imgs_neg.shape)
    #imgs_anchor = imgs_anchor.transpose([0,2,3,1])
    #imgs_pos = imgs_pos.transpose([0,2,3,1])
    #imgs_neg = imgs_neg.transpose([0,2,3,1])
    
    for idx_img in range(batch_size):
        anchor = imgs_anchor[idx_img]
        pos = imgs_pos[idx_img]
        neg = imgs_neg[idx_img]
        print(anchor.shape)
        print(pos.shape)
        print(neg.shape)
        _ShowImg(anchor)
        _ShowImg(pos)
        _ShowImg(neg)
        break
    
    print('data size is {}'.format(reader.GetDataSize()))

In [ ]:
_TestTripletGenerator(reader)

In [ ]:
gen_train = TripletGeneratorSingleID(reader)
gen_test = TripletGeneratorSingleID(reader)

In [ ]:
gen_train

In [ ]:
embedding_model, triplet_model = GetModel()

In [ ]:
for layer in embedding_model.layers[-3:]:
    layer.trainable = True
    
for layer in embedding_model.layers[: -3]:
    layer.trainable = False

In [ ]:
embedding_model.summary()

In [ ]:
batch_size = 16

In [ ]:
triplet_model.compile(loss=None, optimizer=Adam(0.001))

history = triplet_model.fit_generator(gen_train, 
                          validation_data=gen_test,  
                          epochs=1, 
                          verbose=1, 
                            steps_per_epoch=50,
                          validation_steps=5)

In [ ]:
embedding_model.save_weights('/home/sbikevn360/celebrities-recognition/weights/weights_finetune_50_5.h5')

In [ ]:
model = VggFace(
    path = '/home/sbikevn360/celebrities-recognition/weights/weights_finetune_50_5.h5',
                              is_origin = False)

In [ ]:
model.save_weights('/home/sbikevn360/celebrities-recognition/weights/weights_finetune_50_5.h5')

In [ ]:
model.save('/home/sbikevn360/celebrities-recognition/weights/my_model.h5')

In [ ]:
metadata = load_metadata('/home/sbikevn360/celebrities-recognition/face')

In [ ]:
from PIL import Image
import numpy as np
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
import matplotlib.pyplot as plt

In [ ]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

In [ ]:
def findCosineSimilarity(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))
 
def findEuclideanDistance(source_representation, test_representation):
    euclidean_distance = source_representation - test_representation
    euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
    euclidean_distance = np.sqrt(euclidean_distance)
    return euclidean_distance

In [ ]:
embedded = np.zeros((metadata.shape[0], 2622 ))
for i in range(metadata.shape[0]):
    img_emb = model.predict(preprocess_image(metadata[i].image_path()))[0,:]
    embedded[i] = img_emb

In [ ]:
#2h41
# save embedding
np.savetxt('/home/sbikevn360/celebrities-recognition/embedding/embedded_vector.txt', embedded)

In [ ]:
name = []
for i in range(len(metadata)):
    name.append(metadata[i].name)

In [ ]:
len(name)

In [ ]:
np.savetxt('/home/sbikevn360/celebrities-recognition/embedding/name.txt', name,  delimiter=" ", fmt="%s")

In [ ]:
np.savetxt('/home/sbikevn360/celebrities-recognition/embedding/embedded_vector.txt')

In [24]:
os.listdir('./face/Tran Thanh/')

['5.jpg',
 '10.jpg',
 '8.jpg',
 '13.jpg',
 '14.jpg',
 '4.jpg',
 '6.jpg',
 '1.jpg',
 '16.jpg',
 '3.jpg',
 '9.jpg']

In [21]:
rm -rf /face/Tran Thanh/.ipynb_checkpoints

In [22]:
rm -rf /face/Tran Thanh/.ipynb_checkpoints 